In [1]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 47.5 MB/s eta 0:00:0000:010:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install pyarrow fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 44.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.9/579.9 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 36.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
spark=SparkSession.builder.config("spark.driver.memory","12g").config("spark.memory.offHeap.enabled","true") .config("spark.memory.offHeap.size","10g").appName('task3_ml').getOrCreate()

23/05/12 22:09:49 WARN Utils: Your hostname, Adedayos-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.2.27 instead (on interface en0)
23/05/12 22:09:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/12 22:09:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [5]:
file_path = './ml_input_data.parquet/*.parquet'
pyspark_df = spark.read.parquet(file_path, header=True, inferSchema=True)
pyspark_df.show(3)

+------+----------+--------------+---------------------+
|Symbol|Volume_New|vol_moving_avg|adj_close_rolling_med|
+------+----------+--------------+---------------------+
|    AB|  471200.0|          null|                  NaN|
|    AB|  430000.0|      471200.0|           0.03649914|
|    AB|  430000.0|      450600.0|           0.03694973|
+------+----------+--------------+---------------------+
only showing top 3 rows



In [6]:
pandas_df = pyspark_df.toPandas()
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
0,AB,471200.0,NaN,NaN
1,AB,430000.0,471200.0,0.036499
2,AB,430000.0,450600.0,0.036950


In [7]:
#Missing Values
pandas_df.dropna(axis = 0, inplace = True)
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
1,AB,430000.0,471200.000000,0.036499
2,AB,430000.0,450600.000000,0.036950
3,AB,430000.0,443733.333333,0.037400


In [8]:
pandas_df.reset_index(drop = True, inplace= True)
pandas_df.head(3)

,Symbol,Volume_New,vol_moving_avg,adj_close_rolling_med
0,AB,430000.0,471200.000000,0.036499
1,AB,430000.0,450600.000000,0.036950
2,AB,430000.0,443733.333333,0.037400


In [9]:
x_df = pandas_df[['Symbol', 'vol_moving_avg', 'adj_close_rolling_med']]
y_df = pandas_df['Volume_New']

In [10]:
x_df.head(3)

,Symbol,vol_moving_avg,adj_close_rolling_med
0,AB,471200.000000,0.036499
1,AB,450600.000000,0.036950
2,AB,443733.333333,0.037400


In [10]:
%pip install scikit-learn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.7/28.7 MB 48.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 49.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [31]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(x_df,y_df, test_size = 0.2, stratify=x_df[["Symbol"]])

In [14]:
%pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 30.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [32]:
from category_encoders.target_encoder import TargetEncoder
target_encoder = TargetEncoder()
symbol_encoder = target_encoder.fit(x_df['Symbol'],y_df)

In [33]:
x_train['Symbol_2'] =  symbol_encoder.transform(x_train['Symbol']) 
x_test['Symbol_2'] =  symbol_encoder.transform(x_test['Symbol'])

In [34]:
x_train.head(5)

,Symbol,vol_moving_avg,adj_close_rolling_med,Symbol_2
26767136,TSEM,1.402050e+06,13.570000,155168.414253
21703258,HTLD,2.814833e+05,6.521318,389639.840997
36722955,WHLM,2.843333e+03,6.160000,6161.057136
27975754,EME,1.369379e+05,5.688084,447475.404320
13071781,TDS,5.101667e+04,1.135765,347092.493029


In [37]:
x_train.iloc[[0]]

,Symbol,vol_moving_avg,adj_close_rolling_med,Symbol_2
26767136,TSEM,1402050.0,13.57,155168.414253


In [41]:
x_train = x_train.drop('Symbol', axis=1)
x_test = x_test.drop('Symbol', axis=1)
x_train.iloc[[0]]

,vol_moving_avg,adj_close_rolling_med,Symbol_2
26767136,1402050.0,13.57,155168.414253


In [36]:
x_df.head(5)

,Symbol,vol_moving_avg,adj_close_rolling_med
0,AB,471200.000000,0.036499
1,AB,450600.000000,0.036950
2,AB,443733.333333,0.037400
3,AB,440300.000000,0.037400
4,AB,427520.000000,0.037400


In [56]:
y_df.head(5)

0    430000.0
1    430000.0
2    430000.0
3    376400.0
4    364400.0
Name: Volume_New, dtype: float64

In [48]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# ct = ColumnTransformer(transformers= [ ("encoder", OneHotEncoder(),[0])], remainder="passthrough")


In [49]:
# symbol_encoder =  ct.fit(x_df) 

In [38]:
import pickle
filename = './sklearn_models/symbol_target_encoder.sav'
pickle.dump(symbol_encoder, open(filename, 'wb'))

In [50]:
# import numpy as np
# x_train =  symbol_encoder.transform(x_train) 
# x_test =  symbol_encoder.transform(x_test) 

In [ ]:
"""
Linear regression modelling
"""

In [42]:
from sklearn.linear_model import LinearRegression 

In [43]:
LR = LinearRegression()
LR.fit(x_train  ,y_train)

LinearRegression()

In [44]:
filename = './sklearn_models/linear_regression.sav'
pickle.dump(LR, open(filename, 'wb'))

In [45]:
LR.coef_

array([9.59558852e-01, 1.35350983e-17, 4.19390898e-02])

In [47]:
y_pred = LR.predict(x_test)

In [57]:
array_1 = np.array([['AB',440300,0.037400]])
df = pd.DataFrame(array_1, columns=['Symbol', 'vol_moving_avg', 'adj_close_rolling_med'])

df['Symbol_2'] = symbol_encoder.transform(df['Symbol']) 
df = df.drop('Symbol', axis = 1) 
y_pred = LR.predict(df)
y_pred[0] 

431902.7806085313

In [20]:
x_train 

<32940084x8045 sparse matrix of type '<class 'numpy.float64'>'
	with 98309945 stored elements in Compressed Sparse Row format>

In [50]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import math
import numpy as np

In [51]:
MAE = mean_absolute_error(y_test,y_pred) #4.21 

MSE = mean_squared_error(y_test,y_pred) #32.42 

RMSE = math.sqrt(MSE) #5.62

Ytest_Mean = y_test.mean()  #22.85   

print('Mean Absolute Error:', MAE)
print('Mean Squared Error:', MSE)
print('Root Mean Squared Error:', RMSE)
print('Mean:', Ytest_Mean)

def mean_absolute_percentage_error (y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean( np.abs((y_true - y_pred)/y_true)) * 100


MAPE = mean_absolute_percentage_error(y_test,y_pred) #48%

print('Mean Absolute Percentage Error:', MAPE)

print('Scatter Index (RMSE/Mean):', (RMSE/Ytest_Mean) * 100)

Mean Absolute Error: 425591.285141147
Mean Squared Error: 23118865027331.332
Root Mean Squared Error: 4808208.089021453
Mean: 1101972.3155531338
Mean Absolute Percentage Error: inf
Scatter Index (RMSE/Mean): 436.32748492488014


/var/folders/pv/lpp89_3d53328ck1565vv7p40000gn/T/ipykernel_1545/2240912289.py:16: RuntimeWarning: divide by zero encountered in divide
  return np.mean( np.abs((y_true - y_pred)/y_true)) * 100


In [ ]:
"""
Random Forest Regression
"""

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [46]:
RFR = RandomForestRegressor(n_estimators=200,max_depth = 20, random_state=33)

In [ ]:
RFR.fit(x_train,y_train)

In [ ]:
y_pred = RFR.predict(x_test)

In [ ]:
"""
Support Vector Regression
"""

In [23]:
from sklearn.svm import SVR

In [24]:
Regressor_SVR = SVR(kernel="rbf")

In [ ]:
Regressor_SVR.fit(x_train,y_train)